In [1]:
import jax
import jax.numpy as jnp
from genjax.var_elim import var_elim
from genjax import ChoiceMap, gen
import genjax

genjax.pretty()

@gen
def model():
    v = genjax.flip(0.1) @ "v"
    w = genjax.flip(jax.lax.cond(v, lambda: 0.9, lambda: 0.2)) @ "w"
    return w

# display(jax.make_jaxpr(model.source)())
# display(jax.make_jaxpr(model.lower)(ChoiceMap.empty(), ()))
model_elim = var_elim(model, "v")
display(jax.make_jaxpr(model_elim.source)())
true_prob = jnp.exp(model_elim.assess(ChoiceMap.kw(w=1.0), ())[0])
assert jnp.isclose(true_prob, 0.1 * 0.9 + (1 - 0.1) * 0.2)
tr = model_elim.simulate(jax.random.key(1), ())
tr.get_choices()

{ lambda a:bool[2] b:f32[2]; . let
    c:f32[2] = log b
    d:i32[2] = convert_element_type[new_dtype=int32 weak_type=False] a
    _:bool[2] = eq d 0
    e:f32[2] = broadcast_in_dim[
      broadcast_dimensions=()
      shape=(2,)
      sharding=None
    ] 0.2
    _:bool[2] = eq d 1
    f:f32[2] = broadcast_in_dim[
      broadcast_dimensions=()
      shape=(2,)
      sharding=None
    ] 0.9
    g:f32[2] = select_n d e f
    h:bool[] = trace[
      abs_eval=<function initial_style_bind.<locals>.bind.<locals>.wrapped.<locals>._abs_eval at 0x370dde8e0>
      impl=<function initial_style_bind.<locals>.bind.<locals>.wrapped.<locals>._impl at 0x370ddde40>
      in_tree=PyTreeDef((CustomNode(Const[StructStaticMetadata(child_field_names=[], static_fields={'val': 'w'})], []), CustomNode(genjax.genjax.flip.mixture[StructStaticMetadata(child_field_names=[], static_fields={})], []), (*, *)))
      num_consts=0
      out_tree=<function transformation_with_aux2.<locals>.<lambda> at 0x370dde980>
    ]

Static({'w': Choice(v=<jax.Array(False, dtype=bool)>)})